In [ ]:
pip install pandas openpyxl

In [ ]:
#1 Preprocessing

import json
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
import pandas as pd

# Download NLTK data files (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')


def preprocess(text):
    """
    Preprocess the input text by tokenizing, removing non-alphanumeric tokens, 
    converting to lowercase, removing stopwords, and stemming.
    """
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove non-alphanumeric and convert to lowercase
    tokens = [re.sub(r'\W+', '', token).lower() for token in tokens if token.isalnum()]
    
    # Stopword removal
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    
    return tokens

# Read and preprocess documents
corpus = {}
try:
    with open("corpus.jsonl", "r") as f:
        for line in f:
            # Parse the JSON line
            doc = json.loads(line)
            
            # Extract document ID and text (instead of abstract)
            if '_id' in doc and 'text' in doc:
                doc_id = doc['_id']  # Use '_id' as the document identifier
                text = doc['text']  # Use 'text' as the content to preprocess
                
                # Preprocess and store the result
                corpus[doc_id] = preprocess(text)
            else:
                print(f"Skipping document due to missing keys: {doc}")
except FileNotFoundError:
    print("Error: The file 'corpus.jsonl' was not found.")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

# Convert the processed corpus into a list of dictionaries for easy saving into an Excel file
data = []
for doc_id, tokens in corpus.items():
    data.append({"Document ID": doc_id, "Tokens": " ".join(tokens)})

# Create a DataFrame
df = pd.DataFrame(data)

# Save to Excel
excel_filename = "preprocessed_corpus.xlsx"
df.to_excel(excel_filename, index=False)

print(f"Preprocessed data has been saved to {excel_filename}")


In [ ]:
#2 Inverted indexing
import json
from collections import defaultdict

def invertIndex(corpus):
    
    inverted_index = defaultdict(set)  
    
    # Iterate over corpus to add each word to set
    for id, tokens in corpus.items():
        for token in tokens:
            inverted_index[token].add(id)
    
    return inverted_index


inverted_index = invertIndex(corpus)


# Output 
json_data = []
for word, doc_ids in inverted_index.items():
    print(f"{word}: {sorted(doc_ids)}")
    json_data.append({
        "word": word,
        "ids": ", ".join(map(str, sorted(doc_ids)))
    })

with open('inverted_index.json', 'w') as f:
    json.dump(json_data, f, indent=4)
